In [ ]:
import cv2
import numpy as np
import tensorflow as tf
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Conv2D, MaxPooling2D, Flatten, Dense, Dropout
from tensorflow.keras.optimizers import Adam
import os

# Data generators with validation split
train_datagen = ImageDataGenerator(rescale=1./255, validation_split=0.2)

train_generator = train_datagen.flow_from_directory(
    'dataset',
    target_size=(150, 150),
    batch_size=32,
    class_mode='binary',
    subset='training'
)

validation_generator = train_datagen.flow_from_directory(
    'dataset',
    target_size=(150, 150),
    batch_size=32,
    class_mode='binary',
    subset='validation'
)

# Verify class indices
print("Class indices:", train_generator.class_indices)

# Model architecture
model = Sequential([
    Conv2D(32, (3,3), activation='relu', input_shape=(150,150,3)),
    MaxPooling2D(2,2),
    Conv2D(64, (3,3), activation='relu'),
    MaxPooling2D(2,2),
    Flatten(),
    Dense(128, activation='relu'),
    Dropout(0.5),
    Dense(1, activation='sigmoid')
])

model.compile(optimizer=Adam(learning_rate=0.001),
              loss='binary_crossentropy',
              metrics=['accuracy'])

# Model training
model.fit(train_generator, epochs=10, validation_data=validation_generator)

# Save model
model.save('mask_detector_model.h5')

# Face detection setup
face_cascade = cv2.CascadeClassifier(cv2.data.haarcascades + 'haarcascade_frontalface_default.xml')
cap = cv2.VideoCapture(0)

while True:
    ret, frame = cap.read()
    if not ret:
        break
    
    gray = cv2.cvtColor(frame, cv2.COLOR_BGR2GRAY)
    faces = face_cascade.detectMultiScale(gray, 1.1, 4)
    
    for (x, y, w, h) in faces:
        face = frame[y:y+h, x:x+w]
        face = cv2.resize(face, (150, 150))
        face = cv2.cvtColor(face, cv2.COLOR_BGR2RGB)  # Convert to RGB
        face = np.expand_dims(face, axis=0) / 255.0
        
        prediction = model.predict(face)[0][0]
        # Use class indices from generator for correct labeling
        label = 'Mask' if prediction < 0.5 else 'No Mask'
        color = (0, 255, 0) if prediction < 0.5 else (0, 0, 255)
        
        cv2.rectangle(frame, (x, y), (x+w, y+h), color, 2)
        cv2.putText(frame, label, (x, y-10), cv2.FONT_HERSHEY_SIMPLEX, 0.8, color, 2)
    
    cv2.imshow('Face Mask Detector', frame)
    if cv2.waitKey(1) & 0xFF == ord('q'):
        break

cap.release()
cv2.destroyAllWindows()

Found 6043 images belonging to 2 classes.
Found 1510 images belonging to 2 classes.
Class indices: {'with_mask': 0, 'without_mask': 1}


C:\Users\Zeshan\anaconda3\Lib\site-packages\keras\src\layers\convolutional\base_conv.py:107: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)
C:\Users\Zeshan\anaconda3\Lib\site-packages\keras\src\trainers\data_adapters\py_dataset_adapter.py:121: UserWarning: Your `PyDataset` class should call `super().__init__(**kwargs)` in its constructor. `**kwargs` can include `workers`, `use_multiprocessing`, `max_queue_size`. Do not pass these arguments to `fit()`, as they will be ignored.
  self._warn_if_super_not_called()


Epoch 1/10
134/189 ━━━━━━━━━━━━━━━━━━━━ 12s 227ms/step - accuracy: 0.6840 - loss: 1.0823

C:\Users\Zeshan\anaconda3\Lib\site-packages\PIL\Image.py:1056: UserWarning: Palette images with Transparency expressed in bytes should be converted to RGBA images
  warnings.warn(


189/189 ━━━━━━━━━━━━━━━━━━━━ 52s 270ms/step - accuracy: 0.7212 - loss: 0.9133 - val_accuracy: 0.9159 - val_loss: 0.2186
Epoch 2/10
189/189 ━━━━━━━━━━━━━━━━━━━━ 23s 121ms/step - accuracy: 0.8890 - loss: 0.2536 - val_accuracy: 0.9331 - val_loss: 0.1808
Epoch 3/10
189/189 ━━━━━━━━━━━━━━━━━━━━ 23s 122ms/step - accuracy: 0.9341 - loss: 0.1817 - val_accuracy: 0.9272 - val_loss: 0.1852
Epoch 4/10
189/189 ━━━━━━━━━━━━━━━━━━━━ 23s 122ms/step - accuracy: 0.9371 - loss: 0.1601 - val_accuracy: 0.9344 - val_loss: 0.2002
Epoch 5/10
189/189 ━━━━━━━━━━━━━━━━━━━━ 23s 123ms/step - accuracy: 0.9583 - loss: 0.1083 - val_accuracy: 0.9470 - val_loss: 0.1683
Epoch 6/10
189/189 ━━━━━━━━━━━━━━━━━━━━ 23s 124ms/step - accuracy: 0.9671 - loss: 0.0839 - val_accuracy: 0.9417 - val_loss: 0.2027
Epoch 7/10
189/189 ━━━━━━━━━━━━━━━━━━━━ 24s 125ms/step - accuracy: 0.9746 - loss: 0.0741 - val_accuracy: 0.9371 - val_loss: 0.2238
Epoch 8/10
189/189 ━━━━━━━━━━━━━━━━━━━━ 24s 126ms/step - accuracy: 0.9774 - loss: 0.0587 - val

1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 50ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 21ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 21ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 22ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 21ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 21ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 21ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 21ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 21ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 22ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 21ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 22ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 21ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 22ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 21ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 20ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 22ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 21ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 21ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 22ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 21ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 21ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 20ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 22ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 22ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 21ms/step
1/1 ━━━━━━━━